In [28]:
from sklearn.decomposition import PCA
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Use the SVD to create a rank N approximation of the matrix
def rank_n_approximation(mtx, rank_n):
    SVD = np.linalg.svd(mtx, full_matrices=False)
    u, s, v = SVD
    Ar = np.zeros((len(u), len(v)))
    for i in range(rank_n):
        Ar += s[i] * np.outer(u.T[i], v[i])
    return Ar

In [29]:
def run_PCA_analysis(mat):
    pca = PCA(n_components=3)
    mat = np.asarray(mat)
    pca.fit(mat) 

    #Data points projected along the first, second and the third PC
    projectedPrincipal = np.matmul(mat, pca.components_[0])
    projectedSecondary = np.matmul(mat, pca.components_[1])
    projectedTertiary = np.matmul(mat, pca.components_[2])
    
    # print("Printing PC1")
    # print(pca.components_[0])
    pc1 = pca.components_[0]
    # print('')
    # print("Printing PC2")
    pc2 = pca.components_[1]
    # print(pca.components_[1])
    # print('')
    # print("Printing PC3")
    pc3 = pca.components_[2]
    # print(pca.components_[2])
    # print('')
    result_mat = np.matrix([pc1, pc2, pc3])
    print(result_mat)

In [30]:
mat = np.matrix([[1, 4, 3], [5, 2, 3], [1, 2, 4]])

def analyze_PCA_approximations(mat):
    rank = len(mat)
    # Run PCA analysis on original matrix
    run_PCA_analysis(mat)
    # Run PCA analysis on rank n approximations of matrix
    for r in range(1, min(rank + 1, 10)):
        approx = rank_n_approximation(mat, r)
        print('Printing rank ' + str(r) + ' approximation PCA componnts....')
        print('')
        print('')
        run_PCA_analysis(approx)

analyze_PCA_approximations(mat)

[[ 0.95531474 -0.27827157 -0.0996929 ]
 [-0.1993858  -0.85562184  0.47765737]
 [-0.21821789 -0.43643578 -0.87287156]]
Printing rank 1 approximation PCA componnts....


[[ 0.51769453  0.53517292  0.66751952]
 [-0.34077303 -0.58666864  0.73463845]
 [-0.78477137  0.60779096  0.12134265]]
Printing rank 2 approximation PCA componnts....


[[ 0.95282972 -0.29001234 -0.0894895 ]
 [-0.26287923 -0.64123092 -0.72091429]
 [ 0.1516906   0.71043349 -0.6872221 ]]
Printing rank 3 approximation PCA componnts....


[[ 0.95531474 -0.27827157 -0.0996929 ]
 [-0.1993858  -0.85562184  0.47765737]
 [-0.21821789 -0.43643578 -0.87287156]]


In [31]:
def readAndProcessData():
    """
        Function to read the raw text file into a dataframe and keeping the 
        population, gender separate from the genetic data
        
        We also calculate the population mode for each attribute or trait (columns)
        Note that mode is just the most frequently occuring trait
        
        return: dataframe (df), modal traits (modes), population and gender for each individual row
    """
    
    df = pd.read_csv('p4dataset2020.txt', header=None, delim_whitespace=True)
    gender = df[1]
    population = df[2]
    
    df.drop(df.columns[[0, 1, 2]],axis=1,inplace=True)
    modes = np.array(df.mode().values[0,:])
    return df, modes, population, gender
    
df, modes, population, gender = readAndProcessData()

def convertDfToMatrix(df, mode):
    """
        Create a binary matrix (binarized) representing mutations away from mode
        Each row is for an individual, and each column is a trait
        
        binarized_{i,j}= 0 if the $i^{th}$ individual has column 
        $j$’s mode nucleobase for his or her $j^{th}$ nucleobase, 
        and binarized_{i,j}= 1 otherwise
    """
    
    raw_np = df.to_numpy()
    binarized = np.where(raw_np!=modes, 1, 0)
    return binarized

X = convertDfToMatrix(df, modes)


In [36]:
def squarify(M,val):
    (a,b)=M.shape
    if a>b:
        padding=((0,0),(0,a-b))
    else:
        padding=((0,b-a),(0,0))
    return np.pad(M,padding,mode='constant',constant_values=val)

# Pad the square matrix
X = squarify(X, 0)
num = 500
X = X[0:num,0:num]
analyze_PCA_approximations(squarify(X, 0))

[[ 0.05298402 -0.14726595  0.42687565 -0.17365216  0.20161917 -0.36687793
   0.06537557  0.26710644  0.52663418  0.48467975]
 [-0.29471529 -0.10183315  0.28966355 -0.45511213 -0.10519731  0.44803178
  -0.59125253 -0.1060411   0.17703868 -0.08817575]
 [-0.75807111  0.22932478  0.39217514  0.34783197 -0.09626053 -0.14412959
   0.20820073 -0.11641586  0.00336959 -0.10487585]]
Printing rank 1 approximation PCA componnts....


[[ 0.39019819  0.0694362   0.30892675  0.28651085  0.17402819  0.44313415
   0.4706068   0.11454069  0.348161    0.2879925 ]
 [-0.2420679  -0.0357736   0.1673745  -0.77792641 -0.07401941  0.305636
   0.1816763   0.15071801  0.31431326 -0.23137068]
 [-0.74116988 -0.029198    0.14277097  0.30707367  0.11182793  0.40372787
  -0.25650769  0.19194586 -0.02543839  0.23737882]]
Printing rank 2 approximation PCA componnts....


[[ 0.00698794 -0.16281706  0.48048921 -0.22026407  0.18654228 -0.2988735
  -0.00408866  0.2465318   0.543658    0.46214021]
 [-0.41026003 -0.12947563 